In [3]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 37 kB/s 
     |████████████████████████████████| 198 kB 60.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=63352cd574abfe42addcc70fddd40928426a728be9bd76b03de2c4cb1b307a88
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [5]:
import pyspark
import pandas as pd 
import json 
from pandas import json_normalize 
pyspark.__version__

'3.2.1'

In [6]:
from pyspark.sql.functions import monotonically_increasing_id 

In [7]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession\
        .builder\
        .appName('Python Spark basic example')\
        .config('spark.some.config.option', 'some-value')\
        .getOrCreate()

In [65]:
temp_df = spark.read.option("delimiter"," ").csv('test2.csv', header='True') # column 있으면 header = true로 

In [81]:

temp_df = temp_df.distinct()
temp_df.count()

15576

In [83]:
temp_df = temp_df.select("*").withColumn("id", monotonically_increasing_id())
temp_df.show()
temp_df.count()

+--------+---------+------------------+--------+---+
|카테고리|file_path|              소재|  스타일| id|
+--------+---------+------------------+--------+---+
|  청바지|   529856|          ['데님']|    힙합|  0|
|      탑|  1271817|  ['저지', '우븐']|    힙합|  1|
|    짚업|  1212314|          ['우븐']|    힙합|  2|
|    재킷|   496459|          ['우븐']|    힙합|  3|
|      탑|   191577|          ['저지']|    힙합|  4|
|  후드티|   191936|          ['저지']|    힙합|  5|
|    팬츠|   426926|          ['저지']|  클래식|  6|
|    null|    67489|              null|  클래식|  7|
|  스커트|  1216048|          ['시폰']|  클래식|  8|
|    null|   491083|              null|  클래식|  9|
|    코트|   377492|   ['울/캐시미어']|  클래식| 10|
|    팬츠|   329292|          ['우븐']|  톰보이| 11|
|  티셔츠|   519783|          ['저지']|  톰보이| 12|
|    셔츠|  1044024|          ['우븐']|  톰보이| 13|
|    코트|  1129363|['우븐', '플리스']|  웨스턴| 14|
|    재킷|   592109|          ['가죽']|  웨스턴| 15|
|    재킷|   742126|   ['울/캐시미어']|  프레피| 16|
|니트웨어|   778312|          ['니트']|  프레피| 17|
|  티셔츠|   831070|      

15576

In [67]:
temp_df.printSchema()
temp_df.select('소재').show()

root
 |-- 카테고리: string (nullable = true)
 |-- file_path: string (nullable = true)
 |-- 소재: string (nullable = true)
 |-- 스타일: string (nullable = true)
 |-- id: long (nullable = false)

+--------------------+
|                소재|
+--------------------+
|            ['데님']|
|            ['데님']|
|            ['저지']|
|            ['우븐']|
|            ['저지']|
|  ['플리스', '벨벳']|
|  ['플리스', '벨벳']|
|            ['저지']|
|  ['플리스', '벨벳']|
|            ['저지']|
|                null|
|            ['저지']|
|['코듀로이', '린넨']|
|            ['데님']|
|            ['데님']|
|            ['우븐']|
|            ['우븐']|
|            ['우븐']|
|            ['우븐']|
|            ['데님']|
+--------------------+
only showing top 20 rows



In [84]:
for_vectors = temp_df.select('스타일','카테고리','file_path','소재','id')
for_vectors = for_vectors.na.drop("any")
for_vectors.show()

+------+--------+---------+------------------+---+
|스타일|카테고리|file_path|              소재| id|
+------+--------+---------+------------------+---+
|  힙합|  청바지|   529856|          ['데님']|  0|
|  힙합|      탑|  1271817|  ['저지', '우븐']|  1|
|  힙합|    짚업|  1212314|          ['우븐']|  2|
|  힙합|    재킷|   496459|          ['우븐']|  3|
|  힙합|      탑|   191577|          ['저지']|  4|
|  힙합|  후드티|   191936|          ['저지']|  5|
|클래식|    팬츠|   426926|          ['저지']|  6|
|클래식|  스커트|  1216048|          ['시폰']|  8|
|클래식|    코트|   377492|   ['울/캐시미어']| 10|
|톰보이|    팬츠|   329292|          ['우븐']| 11|
|톰보이|  티셔츠|   519783|          ['저지']| 12|
|톰보이|    셔츠|  1044024|          ['우븐']| 13|
|웨스턴|    코트|  1129363|['우븐', '플리스']| 14|
|웨스턴|    재킷|   592109|          ['가죽']| 15|
|프레피|    재킷|   742126|   ['울/캐시미어']| 16|
|프레피|니트웨어|   778312|          ['니트']| 17|
|  펑크|    재킷|    13814|          ['가죽']| 19|
|  펑크|  스커트|   994983|  ['우븐', '메시']| 20|
|  펑크|  래깅스|   558308|      ['스판덱스']| 21|
|  펑크|  티셔츠|   976533|          

In [85]:
from pyspark.sql.functions import col,isnan, when, count
for_vectors.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in for_vectors.columns]
   ).show()

+------+--------+---------+----+---+
|스타일|카테고리|file_path|소재| id|
+------+--------+---------+----+---+
|     0|       0|        0|   0|  0|
+------+--------+---------+----+---+



In [144]:
val=for_vectors.select('카테고리').distinct().rdd.flatMap(lambda x: x).collect()

for col in range(len(val)):
    print(col)
    if val[col] == '-1':
        del val[col]
        break
print(val)

0
1
['후드티', '점프수트', '베스트', '코트', '셔츠', '조거팬츠', '가디건', '팬츠', '패딩', '티셔츠', '청바지', '탑', '짚업', '점퍼', '스커트', '니트웨어', '브라탑', '블라우스', '재킷', '래깅스', '드레스']


In [145]:
from pyspark.sql.functions import lit, when 
for col in val:
  for_vectors=for_vectors.withColumn(f"{col}", when((for_vectors['카테고리'] == col), lit(1))\
                         .otherwise(lit(0)))

In [106]:
for_vectors.show()

+------+--------+------------+---+
|스타일|카테고리|        소재| id|
+------+--------+------------+---+
|레트로|  스커트|    ['실크']|  0|
|레트로|  드레스|    ['데님']|  1|
|레트로|  드레스|    ['우븐']|  2|
|레트로|      -1|          -1|  3|
|레트로|      -1|          -1|  4|
|레트로|    셔츠|    ['우븐']|  5|
|레트로|  드레스|    ['우븐']|  6|
|레트로|      -1|          -1|  7|
|레트로|      -1|          -1|  8|
|레트로|    셔츠|    ['우븐']|  9|
|레트로|  스커트|    ['데님']| 10|
|레트로|    팬츠|['코듀로이']| 11|
|레트로|  드레스|    ['우븐']| 12|
|레트로|블라우스|          -1| 13|
|레트로|  청바지|    ['데님']| 14|
|레트로|  청바지|    ['데님']| 15|
|레트로|      탑|    ['우븐']| 16|
|레트로|  드레스|    ['시폰']| 17|
|레트로|  가디건|    ['니트']| 18|
|레트로|    셔츠|    ['우븐']| 19|
+------+--------+------------+---+
only showing top 20 rows



In [139]:
val=for_vectors.select('스타일').distinct().rdd.flatMap(lambda x: x).collect()
print(val)
for col in range(len(val)):
    print(col)
    if val[col] == '-1':
        del val[col]
        break
print(len(val))
from pyspark.sql.functions import lit, when 
for col in val:
  for_vectors=for_vectors.withColumn(f"{col}", when((for_vectors['스타일'] == col), lit(1))\
                         .otherwise(lit(0)))

['밀리터리', '레트로', '컨트리', '소피스트케이티드', '히피', '모던', '매니시', '클래식', '로맨틱', '리조트']
0
1
2
3
4
5
6
7
8
9
10


In [142]:
val=for_vectors.select('소재').distinct().rdd.flatMap(lambda x: x).collect()

for col in range(len(val)):
    print(col)
    if val[col] == '-1':
        del val[col]
        break
print(len(val))
from pyspark.sql.functions import lit, when 
for col in val:
  for_vectors=for_vectors.withColumn(f"{col}", when((for_vectors['소재'] == str(col)), lit(1))\
                         .otherwise(lit(0)))

0
56


In [146]:
for_vectors.show()

+------+--------+------------+---+--------+------+------+----------------+----+----+------+------+------+------+----------+------------------------+----------------+----------------+----------+----------------+----------------+----------------+----------------+------------+------------------+---------------------+---------------------+------------------+----------------+----------+-----------------------+---------------+-----------------------+----------------+--------+----------------+-----------------------+--------+------------+----------------+------------+--------+--------------------+------------+----------+------------+--------------------+--------------+--------+-------------------------------+--------------+------------------+--------+------------------+----------------+-----------------------+------------------+----------------+--------+----------------+--------+----------------+--------+------------------+--------------------+--------+-------------+------+------------------+

In [86]:
from sklearn.metrics.pairwise import cosine_similarity
result_df = for_vectors.select(for_vectors.columns[:5])
print(result_df)
#cosine_similarity(result_df., result_df.show())

DataFrame[스타일: string, 카테고리: string, file_path: string, 소재: string, id: bigint]


In [157]:
import pyspark.sql.functions as psf 
test_df = temp_df.select(["카테고리","소재","스타일"])
from pyspark.ml.feature import HashingTF, IDF
hashingTF = HashingTF(inputCol="Office_Loc", outputCol="tf")
tf = hashingTF.transform(test_df)

idf = IDF(inputCol="tf", outputCol="feature").fit(tf)
tfidf = idf.transform(tf)

#df = rdd.toDF('카테고리',"소재","스타일").withColumn("Loc", psf.split(psf.regexp_replace("Office_Loc", " ", ""), ','))

IllegalArgumentException: ignored

In [14]:
print(result_df.printSchema())

root
 |-- 스타일: string (nullable = true)
 |-- 카테고리: string (nullable = true)
 |-- 소재: string (nullable = true)
 |-- id: long (nullable = false)

None


In [87]:
from pyspark.sql.functions import concat, col, lit
import pyspark.sql.functions as F
test_df = result_df.withColumn('feature',
                               concat(col('스타일'),lit(" "), col("카테고리"), lit(" "), col("소재")))

#concat(col("스타일"), lit(" "), col("카테고리"), lit(" ", col("소재")))

In [144]:
from pyspark.sql.functions import split, col

df = test_df.select("id","file_path",split(col("feature"), " ").alias("ArrayFeature"))\
          .drop("feature")
df.printSchema()
df.show()

root
 |-- id: long (nullable = false)
 |-- file_path: string (nullable = true)
 |-- ArrayFeature: array (nullable = true)
 |    |-- element: string (containsNull = true)

+---+---------+-----------------------------+
| id|file_path|                 ArrayFeature|
+---+---------+-----------------------------+
|  0|   529856|     [힙합, 청바지, ['데님']]|
|  1|  1271817|    [힙합, 탑, ['저지',, '...|
|  2|  1212314|       [힙합, 짚업, ['우븐']]|
|  3|   496459|       [힙합, 재킷, ['우븐']]|
|  4|   191577|         [힙합, 탑, ['저지']]|
|  5|   191936|     [힙합, 후드티, ['저지']]|
|  6|   426926|     [클래식, 팬츠, ['저지']]|
|  8|  1216048|   [클래식, 스커트, ['시폰']]|
| 10|   377492|[클래식, 코트, ['울/캐시미...|
| 11|   329292|     [톰보이, 팬츠, ['우븐']]|
| 12|   519783|   [톰보이, 티셔츠, ['저지']]|
| 13|  1044024|     [톰보이, 셔츠, ['우븐']]|
| 14|  1129363|  [웨스턴, 코트, ['우븐',,...|
| 15|   592109|     [웨스턴, 재킷, ['가죽']]|
| 16|   742126|[프레피, 재킷, ['울/캐시미...|
| 17|   778312| [프레피, 니트웨어, ['니트']]|
| 19|    13814|       [펑크, 재킷, ['가죽']]|
| 20|   994983|  [펑크, 스커트, ['

In [145]:
from pyspark.ml.feature import HashingTF, IDF
hashingTF = HashingTF(inputCol="ArrayFeature", outputCol="tf")
tf = hashingTF.transform(df)

idf = IDF(inputCol="tf", outputCol="feature").fit(tf)
tfidf = idf.transform(tf)

In [146]:
from pyspark.ml.feature import Normalizer
normalizer = Normalizer(inputCol="feature", outputCol="norm")
data = normalizer.transform(tfidf)

In [147]:
data.show() # 이걸 저장해야하고, 나중에 dot product는 사용자의 입력이 들어오면 시작하는 것이다. 

+---+---------+-----------------------------+--------------------+--------------------+--------------------+
| id|file_path|                 ArrayFeature|                  tf|             feature|                norm|
+---+---------+-----------------------------+--------------------+--------------------+--------------------+
|  0|   529856|     [힙합, 청바지, ['데님']]|(262144,[341,2080...|(262144,[341,2080...|(262144,[341,2080...|
|  1|  1271817|    [힙합, 탑, ['저지',, '...|(262144,[2737,118...|(262144,[2737,118...|(262144,[2737,118...|
|  2|  1212314|       [힙합, 짚업, ['우븐']]|(262144,[36956,20...|(262144,[36956,20...|(262144,[36956,20...|
|  3|   496459|       [힙합, 재킷, ['우븐']]|(262144,[36956,15...|(262144,[36956,15...|(262144,[36956,15...|
|  4|   191577|         [힙합, 탑, ['저지']]|(262144,[94719,16...|(262144,[94719,16...|(262144,[94719,16...|
|  5|   191936|     [힙합, 후드티, ['저지']]|(262144,[94719,13...|(262144,[94719,13...|(262144,[94719,13...|
|  6|   426926|     [클래식, 팬츠, ['저지']]|(262144,[14399,94

In [148]:
data.count()

13986

In [32]:
from pyspark.mllib.linalg.distributed import IndexedRow, IndexedRowMatrix
mat = IndexedRowMatrix(
    data.select("id", "norm")\
        .rdd.map(lambda row: IndexedRow(row.id, row.norm.toArray()))).toBlockMatrix()
dot = mat.multiply(mat.transpose())
dot.toLocalMatrix().toArray()


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.7/dist-packages/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [153]:
from pyspark.sql.types import DoubleType
dot_udf = F.udf(lambda x,y: float(x.dot(y)), DoubleType())
val = data.collect()[-1]
print(val)

result = data.alias("j").join(data.alias("i"), F.col("i.id") == 1)\
    .select(
        F.col("i.id").alias("i"), 
        F.col("j.id").alias("j"), 
        dot_udf("i.norm", "j.norm").alias("dot"))
print(result.show())
print(result.count())

Row(id=15574, file_path='1319026', ArrayFeature=['매니시', '셔츠', "['우븐']"], tf=SparseVector(262144, {36956: 1.0, 90723: 1.0, 176554: 1.0}), feature=SparseVector(262144, {36956: 0.8328, 90723: 3.3037, 176554: 2.9666}), norm=SparseVector(262144, {36956: 0.1843, 90723: 0.7313, 176554: 0.6567}))
+---+---+-------------------+
|  i|  j|                dot|
+---+---+-------------------+
|  1|  0| 0.2459694525181002|
|  1|  1| 1.0000000000000002|
|  1|  2|0.19588486808162775|
|  1|  3|0.28078287046147393|
|  1|  4| 0.5072435932548187|
|  1|  5|0.18446218961519956|
|  1|  6|                0.0|
|  1|  8|                0.0|
|  1| 10|                0.0|
|  1| 11|                0.0|
|  1| 12|                0.0|
|  1| 13|                0.0|
|  1| 14|                0.0|
|  1| 15|                0.0|
|  1| 16|                0.0|
|  1| 17|                0.0|
|  1| 19|                0.0|
|  1| 20|                0.0|
|  1| 21|                0.0|
|  1| 22|                0.0|
+---+---+-----------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
'''
result = data.alias("i").join(data.alias("j"), F.col("i.id") < F.col("j.id"))\
    .select(
        F.col("i.id").alias("i"), 
        F.col("j.id").alias("j"), 
        dot_udf("i.norm", "j.norm").alias("dot")).show()
        '''

#result = data.alias("i").join(data.alias("j").data.select(F.col("i.id").alias("i"),F.col("j.id").alias("j"),
#                                                          dot_udf("i.norm"))

In [141]:
result.sort('dot',ascending=False).collect()[:6]


[Row(i=1, j=1, dot=1.0000000000000002),
 Row(i=1, j=5220, dot=0.8552109830816211),
 Row(i=1, j=7118, dot=0.758860093687738),
 Row(i=1, j=8106, dot=0.7370517791243705),
 Row(i=1, j=12418, dot=0.7370517791243705),
 Row(i=1, j=3983, dot=0.7321820530029697)]

In [154]:
df2 = data.select("id", "ArrayFeature","file_path")
df2.coalesce(1).write.format('json').save('name2.json')

In [137]:
data.show()

+---+-----------------------------+--------------------+--------------------+--------------------+
| id|                 ArrayFeature|                  tf|             feature|                norm|
+---+-----------------------------+--------------------+--------------------+--------------------+
|  0|     [힙합, 청바지, ['데님']]|(262144,[341,2080...|(262144,[341,2080...|(262144,[341,2080...|
|  1|    [힙합, 탑, ['저지',, '...|(262144,[2737,118...|(262144,[2737,118...|(262144,[2737,118...|
|  2|       [힙합, 짚업, ['우븐']]|(262144,[36956,20...|(262144,[36956,20...|(262144,[36956,20...|
|  3|       [힙합, 재킷, ['우븐']]|(262144,[36956,15...|(262144,[36956,15...|(262144,[36956,15...|
|  4|         [힙합, 탑, ['저지']]|(262144,[94719,16...|(262144,[94719,16...|(262144,[94719,16...|
|  5|     [힙합, 후드티, ['저지']]|(262144,[94719,13...|(262144,[94719,13...|(262144,[94719,13...|
|  6|     [클래식, 팬츠, ['저지']]|(262144,[14399,94...|(262144,[14399,94...|(262144,[14399,94...|
|  8|   [클래식, 스커트, ['시폰']]|(262144,[42889,11...|(2621

In [29]:
from pyspark.mllib.linalg.distributed import IndexedRow, IndexedRowMatrix
ac = data.select('norm')
bc = data.select('norm')

mata = IndexedRowMatrix(ac.rdd.map(lambda row: IndexedRow(*row)))
matb = IndexedRowMatrix(bc.rdd.map(lambda row: IndexedRow(*row)))
ma = mata.toBlockMatrix(262144,262144)
mb = matb.toBlockMatrix(262144,262144)

Py4JJavaError: ignored

In [52]:
temp_df.select('스타일').distinct().show()

+----------------+
|          스타일|
+----------------+
|          후드티|
|              -1|
|        점프수트|
|        밀리터리|
|          베스트|
|            코트|
|          레트로|
|            셔츠|
|        조거팬츠|
|          가디건|
|            팬츠|
|            패딩|
|          티셔츠|
|          청바지|
|              탑|
|            짚업|
|          컨트리|
|            점퍼|
|소피스트케이티드|
|            히피|
+----------------+
only showing top 20 rows



In [61]:
val=temp_df.select('스타일').distinct()
val.show()

+----------------+
|          스타일|
+----------------+
|          후드티|
|              -1|
|        점프수트|
|        밀리터리|
|          베스트|
|            코트|
|          레트로|
|            셔츠|
|        조거팬츠|
|          가디건|
|            팬츠|
|            패딩|
|          티셔츠|
|          청바지|
|              탑|
|            짚업|
|          컨트리|
|            점퍼|
|소피스트케이티드|
|            히피|
+----------------+
only showing top 20 rows



In [ ]:
data = '''{"이미지 정보":{"이미지 식별자":93,"이미지 높이":800,"이미지 파일명":"Dahong_415_01.jpg","이미지 너비":800},"데이터셋 정보":{"파일 생성일자":"2020-08-03 01:52:25","데이터셋 상세설명":{"렉트좌표":{"아우터":[{}],"하의":[{}],"원피스":[{}],"상의":[{"X좌표":226.5,"Y좌표":91.5,"가로":365,"세로":482}]},"폴리곤좌표":{"아우터":[{}],"하의":[{}],"원피스":[{}],"상의":[{"Y좌표44":59.0,"Y좌표43":129.0,"Y좌표40":99.0,"Y좌표42":119.0,"Y좌표41":110.0,"X좌표39":342.0,"X좌표38":310.0,"X좌표37":286.0,"X좌표36":270.0,"X좌표35":262.0,"X좌표34":251.0,"X좌표33":242.0,"X좌표32":230.0,"X좌표31":227.0,"X좌표30":250.0,"Y좌표37":127.0,"Y좌표36":161.0,"X좌표8":574.0,"Y좌표39":109.0,"X좌표9":576.0,"Y좌표38":113.0,"Y좌표33":298.0,"Y좌표32":350.0,"Y좌표35":200.0,"Y좌표34":249.0,"Y좌표31":391.0,"Y좌표30":393.0,"X좌표44":534.0,"X좌표2":448.0,"X좌표43":429.0,"X좌표3":467.0,"X좌표42":411.0,"X좌표41":383.0,"X좌표1":438.0,"X좌표40":361.0,"X좌표6":546.0,"X좌표7":568.0,"X좌표4":484.0,"X좌표5":513.0,"Y좌표26":490.0,"Y좌표25":544.0,"Y좌표28":404.0,"Y좌표9":254.0,"Y좌표27":443.0,"Y좌표22":571.0,"Y좌표21":570.0,"Y좌표24":563.0,"Y좌표23":574.0,"Y좌표20":564.0,"X좌표19":505.0,"X좌표18":558.0,"X좌표17":592.0,"X좌표16":586.0,"X좌표15":575.0,"X좌표14":576.0,"X좌표13":572.0,"X좌표12":576.0,"X좌표11":572.0,"Y좌표4":95.0,"X좌표10":576.0,"Y좌표3":92.0,"Y좌표2":116.0,"Y좌표1":125.0,"Y좌표8":200.0,"Y좌표7":154.0,"Y좌표29":399.0,"Y좌표6":113.0,"Y좌표5":104.0,"Y좌표15":481.0,"Y좌표14":457.0,"Y좌표17":539.0,"Y좌표16":514.0,"Y좌표11":314.0,"Y좌표10":282.0,"Y좌표13":405.0,"Y좌표12":367.0,"X좌표29":287.0,"X좌표28":303.0,"X좌표27":293.0,"X좌표26":278.0,"X좌표25":266.0,"X좌표24":285.0,"X좌표23":312.0,"X좌표22":375.0,"X좌표21":409.0,"X좌표20":466.0,"Y좌표19":563.0,"Y좌표18":555.0}]},"라벨링":{"스타일":[{"스타일":"모던"}],"아우터":[{}],"하의":[{}],"원피스":[{}],"상의":[{"기장":"노멀","색상":"네이비","서브색상":"실버","카테고리":"블라우스","디테일":["비즈","패치워크"],"소매기장":"7부소매","소재":["시폰","실크"],"프린트":["그래픽"],"넥라인":"라운드넥","핏":"루즈"}]}},"파일 번호":93,"파일 이름":"Dahong_415_01.jpg"}}'''

In [ ]:
data = '''{"이미지 정보":{"이미지 식별자":1348316,"이미지 높이":1049,"이미지 파일명":"casldkfj-(268).jpg","이미지 너비":800},"데이터셋 정보":{"파일 생성일자":"2020-12-08 13:10:10","데이터셋 상세설명":{"렉트좌표":{"아우터":[{}],"하의":[{"X좌표":227.5,"Y좌표":287.5,"가로":247,"세로":618}],"원피스":[{}],"상의":[{"X좌표":194.5,"Y좌표":5.5,"가로":318,"세로":399}]},"폴리곤좌표":{"아우터":[{}],"하의":[{"Y좌표9":499.0,"Y좌표20":329.0,"X좌표19":375.0,"X좌표18":451.0,"X좌표17":461.0,"X좌표16":475.0,"X좌표15":450.0,"X좌표14":435.0,"X좌표13":407.0,"X좌표12":354.0,"X좌표11":360.0,"Y좌표4":906.0,"X좌표10":367.0,"Y좌표3":687.0,"Y좌표2":450.0,"Y좌표1":329.0,"Y좌표8":676.0,"Y좌표7":689.0,"Y좌표6":870.0,"Y좌표5":897.0,"Y좌표15":718.0,"Y좌표14":854.0,"X좌표8":317.0,"Y좌표17":364.0,"X좌표9":353.0,"Y좌표16":436.0,"Y좌표11":812.0,"Y좌표10":665.0,"Y좌표13":870.0,"Y좌표12":848.0,"X좌표2":228.0,"X좌표3":234.0,"X좌표20":247.0,"X좌표1":248.0,"X좌표6":336.0,"Y좌표19":288.0,"X좌표7":326.0,"Y좌표18":311.0,"X좌표4":259.0,"X좌표5":300.0}],"원피스":[{}],"상의":[{"X좌표8":254.0,"X좌표9":259.0,"X좌표2":220.0,"X좌표3":195.0,"X좌표1":282.0,"X좌표6":219.0,"X좌표7":235.0,"X좌표4":196.0,"X좌표5":223.0,"Y좌표26":33.0,"Y좌표25":57.0,"Y좌표9":332.0,"Y좌표27":9.0,"Y좌표22":15.0,"Y좌표21":53.0,"Y좌표24":63.0,"Y좌표23":38.0,"Y좌표20":82.0,"X좌표19":506.0,"X좌표18":513.0,"X좌표17":453.0,"X좌표16":397.0,"X좌표15":395.0,"X좌표14":383.0,"X좌표13":372.0,"X좌표12":361.0,"X좌표11":355.0,"Y좌표4":291.0,"X좌표10":299.0,"Y좌표3":118.0,"Y좌표2":75.0,"Y좌표1":6.0,"Y좌표8":342.0,"Y좌표7":405.0,"Y좌표6":377.0,"Y좌표5":305.0,"Y좌표15":288.0,"Y좌표14":286.0,"Y좌표17":320.0,"Y좌표16":302.0,"Y좌표11":367.0,"Y좌표10":349.0,"Y좌표13":316.0,"Y좌표12":347.0,"X좌표27":283.0,"X좌표26":303.0,"X좌표25":349.0,"X좌표24":359.0,"X좌표23":379.0,"X좌표22":402.0,"X좌표21":449.0,"X좌표20":485.0,"Y좌표19":148.0,"Y좌표18":269.0}]},"라벨링":{"스타일":[{}],"아우터":[{}],"하의":[{"색상":"베이지"}],"원피스":[{}],"상의":[{"색상":"브라운","소매기장":"긴팔"}]}},"파일 번호":1348316,"파일 이름":"casldkfj-(268).jpg"}}'''

In [ ]:
info = json.loads(data)

df = pd.DataFrame(json_normalize(info['데이터셋 정보'])) #Results contain the required data
#df = pd.DataFrame(json_normalize(info['데이터셋 정보']['데이터셋 상세설명']['라벨링'])) 

In [ ]:
temp=df['데이터셋 상세설명.라벨링.하의']
style = df['데이터셋 상세설명.라벨링.스타일']
if '스타일' in style[0][0].keys():
    print(style[0][0]['스타일'])
else:
    print(-1)

-1


In [ ]:
style = df['데이터셋 상세설명.라벨링.스타일']
style[0][0]['스타일']

'모던'

In [ ]:
result_df = pd.DataFrame(index=range(0,0),columns={'카테고리', '스타일', 'file_path', '디테일'})

In [ ]:
options = ['데이터셋 상세설명.라벨링.아우터','데이터셋 상세설명.라벨링.하의','데이터셋 상세설명.라벨링.원피스','데이터셋 상세설명.라벨링.상의']

style = df['데이터셋 상세설명.라벨링.스타일']
file_path = str(df['파일 번호']).split()[1]
temp2 = []
for i in options:
        temp = df[i]
        print(temp)
        if temp[0][0] != {}:
            temp_df = []
            keys = df[i][0][0].keys()
            print(keys)
            if '카테고리' in keys:

                temp_df.append(df[i][0][0]['카테고리'])
            else:
                temp_df.append(None)
            if '디테일' in keys:

                temp_df.append(df[i][0][0]['디테일'][0])
            else:
                temp_df.append(None)
            if '스타일' in keys:

                temp_df.append(style[0][0]['스타일'])
            else:
                temp_df.append(None)
            
            temp_df.append(file_path)
            print(temp_df)
            temp2.append(temp_df)
            one_result = pd.DataFrame(temp2,columns=['카테고리', '스타일', '디테일', 'file_path' ])
            print(one_result)
            result_df = result_df.append(one_result) # 업데이트를 해줘야한다.

0    [{}]
Name: 데이터셋 상세설명.라벨링.아우터, dtype: object
0    [{}]
Name: 데이터셋 상세설명.라벨링.하의, dtype: object
0    [{}]
Name: 데이터셋 상세설명.라벨링.원피스, dtype: object
0    [{'기장': '노멀', '색상': '네이비', '서브색상': '실버', '카테고리...
Name: 데이터셋 상세설명.라벨링.상의, dtype: object
dict_keys(['기장', '색상', '서브색상', '카테고리', '디테일', '소매기장', '소재', '프린트', '넥라인', '핏'])
['블라우스', '비즈', None, '93']
   카테고리 스타일   디테일 file_path
0  블라우스  비즈  None        93


In [ ]:
result_df

,카테고리,디테일,스타일,file_path
0,블라우스,None,비즈,93


In [ ]:
result_df.iloc[0]

카테고리                                        드레스
디테일                                         레트로
스타일                                         비대칭
file_path    0    278
Name: 파일 번호, dtype: int64
Name: 0, dtype: object

In [ ]:
result_d = pd.read_csv("test.csv", sep=' ', engine='python')

In [ ]:
result_d

,스타일,디테일,카테고리,file_path,소재
0,레트로,NaN,스커트,784954,['실크']
1,레트로,NaN,드레스,1074386,['데님']
2,레트로,NaN,드레스,709363,['우븐']
3,레트로,NaN,-1,486896,-1
4,레트로,NaN,-1,486896,-1
...,...,...,...,...,...
2128,히피,NaN,드레스,597327,-1
2129,히피,NaN,스커트,1280834,['우븐']
2130,히피,NaN,스커트,1280834,['우븐']
2131,히피,NaN,티셔츠,1280834,['니트']


In [ ]:
result_d.drop(columns= ['디테일'], inplace=True)

In [ ]:
from ast import literal_eval

result_d['소재'] = result_d['소재'].apply(literal_eval)

In [ ]:
result_d.iloc[2]['소재']

['우븐']

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity 
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords 
import nltk
nltk.download('stopwords') 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
temp_df

['블라우스', '비즈', '모던', 0    93
 Name: 파일 번호, dtype: int64]

In [ ]:
'''from re import IGNORECASE
options = ['데이터셋 상세설명.라벨링.아우터','데이터셋 상세설명.라벨링.하의','데이터셋 상세설명.라벨링.원피스','데이터셋 상세설명.라벨링.상의']
#style = df['데이터셋 상세설명.라벨링.스타일']
#temp_df = pd.DataFrame(index=range(0,0),columns={'카테고리', '스타일', 'file_path', '디테일'})
temp_df = []
#temp_df['카테고리'] = ''
#temp_df['스타일'] = ''
#temp_df['file_path'] = ''
#temp_df['디테일'] = ''
style = df['데이터셋 상세설명.라벨링.스타일']
file_path = df['파일 번호']
print(file_path)
for i in options:
  temp = df[i]
  print(temp)
  if temp[0][0] != {}:
    temp_df['카테고리'] = df[i][0][0]['카테고리']
    temp_df['디테일'] = df[i][0][0]['디테일'][0]
    temp_df['스타일'] = style[0][0]['스타일']
    temp_df['file_path'] = file_path
    
    one_result = pd.DataFrame(temp_df)
    print(one_result)
    result_df.append(one_result)'''

In [1]:
from pyspark import SparkContext
sc =SparkContext()
rdd = sc.parallelize([[1, "Delhi, Mumbai, Gandhinagar"],[2, " Delhi, Mandi"], [3, "Hyderbad, Jaipur"]])

In [4]:
documents = rdd.map(lambda l: l[1].replace(" ", "").split(","))
print(documents)
from pyspark.mllib.feature import HashingTF, IDF
hashingTF = HashingTF()
tf = hashingTF.transform(documents)

PythonRDD[2] at RDD at PythonRDD.scala:53


In [7]:
tf.cache()
idf = IDF().fit(tf)
tfidf = idf.transform(tf)

In [8]:
from pyspark.mllib.feature import Normalizer
labels = rdd.map(lambda l: l[0])
features = tfidf

normalizer = Normalizer()
data = labels.zip(normalizer.transform(features))


In [10]:
from pyspark.mllib.linalg.distributed import IndexedRowMatrix
data.cartesian(data)\
    .map(lambda l: ((l[0][0], l[1][0]), l[0][1].dot(l[1][1])))\
    .sortByKey()\
    .collect()

[((1, 1), 1.0),
 ((1, 2), 0.10794633570596117),
 ((1, 3), 0.0),
 ((2, 1), 0.10794633570596117),
 ((2, 2), 1.0),
 ((2, 3), 0.0),
 ((3, 1), 0.0),
 ((3, 2), 0.0),
 ((3, 3), 1.0000000000000002)]